In [145]:
import pandas as pd
import re
from collections import defaultdict
from gensim.models.phrases import  Phrases, Phraser
import warnings
warnings.filterwarnings("ignore")

In [146]:
simpsons_data = pd.read_csv("~/Downloads/simpsons_dataset.csv", sep = ",")
simpsons_data.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [147]:
simpsons_data.isna().sum()

raw_character_text    17814
spoken_words          26459
dtype: int64

In [148]:
new_simpsons =simpsons_data.dropna().reset_index(drop = True)

In [149]:
clean_data = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in new_simpsons['spoken_words'])

In [150]:
clean_simp = pd.DataFrame({'clean': clean_data})
clean_simp = clean_simp.dropna().drop_duplicates()
clean_simp.shape

(119516, 1)

Bigram we are using gensim phrases package to automatically detect common phrases(bigrams) from a list of sentences. Using the bigram model to catch or capture words like "mr_burns" or "bart_simpson"!

In [151]:
sentence = [row.split() for row in clean_simp['clean']]

Creates the relevant phrases from the list of sentences:

In [152]:
phrases = Phrases(sentence, min_count=30, progress_per=10000)

In [153]:
bigram = Phraser(phrases)

Tokenizing sentences

In [154]:
sentences_ = bigram[sentence]

In [155]:
word_frequency = defaultdict(int)
for sentence in sentences_:
    for i in sentence:
        word_frequency[i] += 1
len(word_frequency)

40991

getting frequent words from the documents

In [156]:
sorted(word_frequency, key=word_frequency.get, reverse=True)[:10]

['the', 'you', 'i', 'a', 'to', 'and', 'of', 'it', 'my', 'that']

training the model using the gensim word2vec model

In [157]:
import multiprocessing
from gensim.models import Word2Vec

In [158]:
cores = multiprocessing.cpu_count() #count the number of cores in a computer
cores

4

In [159]:
w2v_model = Word2Vec(min_count=20, #ignores all words with total absolute frequency lower than this - (2, 100)
                     window=6,
                     size=300, #dimensionality of the feature vectors. - (50, 300)
                     sample=6e-5, #The threshold for configuring which higher -frequency words are randomly downsampled. Highly influencial. -(0, 1e-5)
                     alpha=0.03, #The initial learning rate -(0.01, 0.05)
                     min_alpha=0.0007, #learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00
                     negative=20, #if > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0,negative sampling is used. - (5, 20)
                     workers=cores-1)

In [160]:
w2v_model.build_vocab(sentences_, progress_per=10000)

In [161]:
w2v_model.train(sentences_, total_examples = w2v_model.corpus_count, epochs=30, report_delay=1)

(12599243, 37627050)

In [162]:
w2v_model.init_sims(replace=True)

In [163]:
w2v_model.wv.most_similar(positive=['homer'])

[('marge', 0.5678645372390747),
 ('dad', 0.46416664123535156),
 ('homie', 0.45369595289230347),
 ('becky', 0.40321457386016846),
 ('bart', 0.3931138813495636),
 ('gee', 0.3921900987625122),
 ('abe', 0.38195741176605225),
 ('mom', 0.3782440423965454),
 ('son', 0.3680479824542999),
 ('honey', 0.36179277300834656)]

checking for similarities between words

In [164]:
w2v_model.similarity("mom", "dad")

0.7335284764817366

checking for words that did not match from an array of words

In [165]:
w2v_model.doesnt_match(['good','smooth','cute'])

'smooth'